## Configuration

In [37]:
### Configuration
# Import dependencies
import os
from collections import Counter
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

## Log In

In [38]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Read in current song candidates

In [67]:
# Get playlist IDs
playlistID1 = getPlaylistID(credentials, '2023')
playlistID2 = getPlaylistID(credentials, '2023 Round 2')

# Read in CurrentYearSongCandidates playlist
playlistTracks1 = fetchTracksFromPlaylist(credentials, playlistID1)
playlistTracks2 = fetchTracksFromPlaylist(credentials, playlistID2)
# Merge dfs
allNewTracks = pd.concat([playlistTracks1, playlistTracks2], axis=0, ignore_index=True)
allNewTracks

,artistName,artistID,trackName,release_date,songURI
0,070 Shake,12Zk1DFhCbHY6v3xep2ZjI,Skin and Bones,2022-06-03,spotify:track:4CZgQq8KJYWGh1bM3sqGmd
1,070 Shake,12Zk1DFhCbHY6v3xep2ZjI,Wine & Spirits,2022-06-03,spotify:track:02iwkyyf6rSZPAe0NQOcjL
2,070 Shake,12Zk1DFhCbHY6v3xep2ZjI,Come Back Home,2022-06-03,spotify:track:1jiOVoiDZdtHiaF5W9XqWN
3,Pluralone,5jQsMNuCW0iZeFTz6YUK8K,Claw Your Way Out,2022-01-27,spotify:track:3HjohYDsYZg2KLhYjto8Hq
4,Ruby Red,2f0NSj1t2L6JowHINXCFb6,No Feeling Is Forever,2022-06-10,spotify:track:68V71NzNC6GRaCaE9S8bcO
...,...,...,...,...,...
802,Herencia de Patrones,1Q6SZxTvaE3HhslV0iXbI6,Brinco En Accion,2022-08-05,spotify:track:3cVBrj8wiMydmzwmqy9du8
803,Twiddle,0hNjIdUHXWhd0dilzi6c12,Beautiful,2022-08-12,spotify:track:4hGypxSXjK9uc3ZwrVr7Lf
804,Twiddle,0hNjIdUHXWhd0dilzi6c12,Distance Makes The Heart,2022-08-12,spotify:track:052VxI8Pg4kCOhdJLk60Cb
805,Anders Beck,12GKG5x2TgNA1zZzigrvtu,Distance Makes The Heart,2022-08-12,spotify:track:052VxI8Pg4kCOhdJLk60Cb


## Read in old playlists

In [68]:
# Read in all old playlists
pl2018 = getPlaylistID(credentials, "Summer 2018")
plt2018 = fetchTracksFromPlaylist(credentials, pl2018)
pl2019 = getPlaylistID(credentials, "Summer 2019")
plt2019 = fetchTracksFromPlaylist(credentials, pl2019)
pl2020 = getPlaylistID(credentials, "Endless Summer 2020")
plt2020 = fetchTracksFromPlaylist(credentials, pl2020)
pl2021 = getPlaylistID(credentials, "Endless Summer 2021")
plt2021 = fetchTracksFromPlaylist(credentials, pl2021)
pl2022 = getPlaylistID(credentials, "Endless Summer 2022")
plt2022 = fetchTracksFromPlaylist(credentials, pl2022)
# Merge dfs
allOldTracks = pd.concat([plt2018, plt2019, plt2020, plt2021, plt2022], axis=0, ignore_index=True)

## Filter out non-qualifiers

In [69]:
# Filter out any songs that have already appeared in old playlists
# Based on songURI alone
newFiltered = allNewTracks.loc[~allNewTracks['songURI'].isin(allOldTracks['songURI'])]
print(len(newFiltered))
# Based on artist ID + trackName, which is more exact
newFiltered['combined'] = newFiltered['artistID']+newFiltered['songName']
allOldTracks['combined'] = allOldTracks['artistID']+allOldTracks['songName']
newFiltered = newFiltered.loc[~newFiltered['combined'].isin(allOldTracks['combined'])]
print(len(newFiltered))

# Filter out any songs older than 5 years
dateMin = datetime((datetime.today() - relativedelta(years=5)).year, 3, 17)
newFiltered['release_date'] = pd.to_datetime(newFiltered['release_date'])
newFiltered = newFiltered.loc[newFiltered['release_date']>dateMin].reset_index(drop=True)
print(len(newFiltered))

# Sort by frequency of artist, artist
# Get frequency of artists in previous summers
frequency = pd.DataFrame(sorted(Counter(allOldTracks['artistID']).items(), key=lambda x:x[1], reverse=True))
frequency.columns = ['artistID', 'artistAppearances']

# Merge with new 
newFiltered = newFiltered.merge(frequency, how='left', on='artistID')
newFiltered['artistAppearances'] = newFiltered['artistAppearances'].fillna(0)

# Sort
newFiltered = newFiltered.sort_values(by=['artistAppearances', 'artistID', 'release_date'], ascending=False, ignore_index=True)

# Drop duplicates
newFiltered = newFiltered.drop_duplicates(subset='songURI', ignore_index=True)
newFiltered = newFiltered.drop_duplicates(subset=['artistID', 'songName'], ignore_index=True)
print(len(newFiltered))

803
797
785
531


/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_43637/2846254092.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newFiltered['combined'] = newFiltered['artistID']+newFiltered['trackName']


## Create new playlist

In [70]:
# Warning: This will make a new playlist
playlistNew = getPlaylistID(credentials, '2023 Cull')
# Add songs to the playlist
addToPlaylist(credentials, newFiltered['songURI'].to_list(), playlistNew)